In [1]:
"""
In this experiment, we decode a Shor's nine-qubit quantum error correcting code.
We show decoding of the Shor's nine-qubit code using Dephasing DMRG --
our own built-in DMRG-like optimisation algorithm to solve the main component problem --
the problem of finding a computational basis state cotributing the most to a given state.
"""

"\nIn this experiment, we decode a Shor's nine-qubit quantum error correcting code.\nWe show decoding of the Shor's nine-qubit code using Dephasing DMRG --\nour own built-in DMRG-like optimisation algorithm to solve the main component problem --\nthe problem of finding a computational basis state cotributing the most to a given state.\n"

In [2]:
import numpy as np
import qecstruct as qec

from decoding import IDENTITY, XOR_BULK, XOR_LEFT, XOR_RIGHT, SWAP
from decoding import bin_vec_to_dense, ConstraintString, get_constraint_sites, apply_parity_constraints, get_codewords, prepare_codewords, apply_bias_channel, decode, linear_code_checks

from mdopt.mps.utils import create_simple_product_state, create_custom_product_state
from mdopt.utils.utils import mpo_to_matrix

In [3]:
# Fixing a random seed
SEED = 123

tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]

In [4]:
def linear_qcode_checks(code: qec.CssCode) -> list[tuple[np.ndarray]]:
    """
    Given a CSS code, returns a list of its checks, where each check
    is represented as a list of indices of the bits adjacent to it.

    Parameters
        code :
            CSS code object.

    Returns
        checks : list of lists
            List of checks.
    """

    parity_matrix_x = code.x_stabs_binary()
    array_x = np.zeros((parity_matrix_x.num_rows(), parity_matrix_x.num_columns()), dtype=int)
    for row, cols in enumerate(parity_matrix_x.rows()):
        for col in cols:
            array_x[row, col] = 1
    
    parity_matrix_z = code.z_stabs_binary()
    array_z = np.zeros((parity_matrix_z.num_rows(), parity_matrix_z.num_columns()), dtype=int)
    for row, cols in enumerate(parity_matrix_z.rows()):
        for col in cols:
            array_z[row, col] = 1

    return [np.nonzero(row)[0] for row in array_x], [np.nonzero(row)[0] for row in array_z]

In [5]:
def get_qconstraint_sites(code: qec.CssCode) -> list[int]:
    """
    Returns the list of MPS sites where the logical constraints should be applied.

    Parameters
        code : :class:`qecstruct.LinearCode`
            Linear code object.

    Returns
        strings : list[int]
            List of MPS sites.
    """

    sites_x, sites_z = linear_qcode_checks(code)

    check_x_degree = len(sites_x[0])
    constraints_strings_x = []

    check_z_degree = len(sites_z[0])
    constraints_strings_z = []

    for sites in sites_x:

        # Retreiving the sites indices where we apply the "bulk"/"boundary" XOR tensors.
        xor_left_sites_x = [sites[0]]
        xor_bulk_sites_x = [sites[i] for i in range(1, check_x_degree - 1)]
        xor_right_sites_x = [sites[-1]]

        # Retreiving the sites indices where we apply the SWAP tensors.
        swap_sites_x = list(range(sites[0] + 1, sites[-1]))
        for k in range(1, check_x_degree - 1):
            swap_sites_x.remove(sites[k])

        constraints_strings_x.append(
            [xor_left_sites_x, xor_bulk_sites_x, swap_sites_x, xor_right_sites_x]
        )

    for sites in sites_z:

        # Retreiving the sites indices where we apply the "bulk"/"boundary" XOR tensors.
        xor_left_sites_z = [sites[0]]
        xor_bulk_sites_z = [sites[i] for i in range(1, check_z_degree - 1)]
        xor_right_sites_z = [sites[-1]]

        # Retreiving the sites indices where we apply the SWAP tensors.
        swap_sites_z = list(range(sites[0] + 1, sites[-1]))
        for k in range(1, check_z_degree - 1):
            swap_sites_z.remove(sites[k])

        constraints_strings_z.append(
            [xor_left_sites_z, xor_bulk_sites_z, swap_sites_z, xor_right_sites_z]
        )

    return constraints_strings_x, constraints_strings_z

In [6]:
code = qec.shor_code()

In [7]:
code

X stabilizers:
[0, 1, 2, 3, 4, 5]
[3, 4, 5, 6, 7, 8]
Z stabilizers:
[0, 1]
[1, 2]
[3, 4]
[4, 5]
[6, 7]
[7, 8]

In [8]:
code.x_stabs_binary()

[0, 1, 2, 3, 4, 5]
[3, 4, 5, 6, 7, 8]

In [9]:
code.z_stabs_binary()

[0, 1]
[1, 2]
[3, 4]
[4, 5]
[6, 7]
[7, 8]

In [10]:
sites_x, sites_z = linear_qcode_checks(code)

In [11]:
print(sites_x)
print(sites_z)

[array([0, 1, 2, 3, 4, 5]), array([3, 4, 5, 6, 7, 8])]
[array([0, 1]), array([1, 2]), array([3, 4]), array([4, 5]), array([6, 7]), array([7, 8])]


In [12]:
qcode_constraint_sites = get_qconstraint_sites(code)

In [13]:
qcode_constraint_sites

([[[0], [1, 2, 3, 4], [], [5]], [[3], [4, 5, 6, 7], [], [8]]],
 [[[0], [], [], [1]],
  [[1], [], [], [2]],
  [[3], [], [], [4]],
  [[4], [], [], [5]],
  [[6], [], [], [7]],
  [[7], [], [], [8]]])

In [15]:
code.x_logicals_binary()

[0, 1, 2]

In [ ]:
string = "000000000000000000"
error = create_custom_product_state(string=string, form="Right-canonical")
state_to_contract = create_simple_product_state(num_sites=18, which="+")
logical_state = create_simple_product_state(num_sites=2, which="0")